In [1]:
# Shape file calculations

import geopandas as gpd
import pandas as pd


neighborhoods_shape = gpd.read_file('geo_export_8ca4e2dd-f403-4b03-b04e-5c77baf2e976.shp') #neighborhoods in City
neighborhoods_shape = neighborhoods_shape.rename(columns={'community': 'pri_neigh'})

parks_shape = gpd.read_file('geo_export_75f4866e-9d62-4c2d-854f-9c37099bbbeb.shp') #parks in Cook County

municipalties_shape = gpd.read_file('2020_CMAP_Municipalities.shp')
cbo_shape = municipalties_shape.loc[municipalties_shape['MUNI'].isin(['Cicero', 'Oak Park', 'Berwyn'])] # suburbs
cbo_shape = cbo_shape.rename(columns={'MUNI': 'pri_neigh'})


crs_projected = 'EPSG:26916' # Best UTM zone for Chicago
neighborhoods_shape = neighborhoods_shape.to_crs(crs_projected)
parks_shape = parks_shape.to_crs(crs_projected)
cbo_shape = cbo_shape.to_crs(crs_projected)

combined_gdf = gpd.GeoDataFrame(pd.concat([neighborhoods_shape, cbo_shape], ignore_index=True, sort=False))
    
combined_gdf['geometry'] = combined_gdf.geometry
parks_shape['geometry'] = parks_shape.geometry
    
intersection_areas = gpd.overlay(combined_gdf, parks_shape, how='intersection')
intersection_areas['intersection_area'] = intersection_areas.geometry.area

area_by_neighborhood = intersection_areas.groupby('pri_neigh')['intersection_area'].sum()
combined_gdf['total_area'] = combined_gdf.geometry.area

park_coverage = (area_by_neighborhood / combined_gdf.set_index('pri_neigh')['total_area'] * 100).reset_index()

park_coverage.columns = ['Neighborhood Name', 'park_coverage_percentage']
park_coverage['Neighborhood Name'] = park_coverage['Neighborhood Name'].str.title()
park_coverage.loc[park_coverage['Neighborhood Name'] == 'Lower West Side', 'Neighborhood Name'] = 'Pilsen'
park_coverage.loc[park_coverage['Neighborhood Name'] == 'South Lawndale', 'Neighborhood Name'] = 'Little Village'
park_coverage.loc[park_coverage['Neighborhood Name'] == 'North Lawndale', 'Neighborhood Name'] = 'Lawndale'
park_coverage.loc[park_coverage['Neighborhood Name'] == 'River North', 'Neighborhood Name'] = 'Near North Side'
park_coverage.loc[park_coverage['Neighborhood Name'] == 'Sauganash,Forest Glen', 'Neighborhood Name'] = 'Forest Glen'
park_coverage.loc[park_coverage['Neighborhood Name'] == 'Ohare', 'Neighborhood Name'] = 'O\'Hare'
park_coverage.loc[park_coverage['Neighborhood Name'] == 'Mckinley Park', 'Neighborhood Name'] = 'McKinley Park'


print(park_coverage)
park_coverage.to_csv('neighborhood_park_coverage.csv', index=False)

   Neighborhood Name  park_coverage_percentage
0        Albany Park                  2.733573
1     Archer Heights                  1.756930
2      Armour Square                  2.816747
3            Ashburn                  1.946591
4     Auburn Gresham                  2.498760
..               ...                       ...
75         West Lawn                  1.864847
76      West Pullman                  2.793776
77        West Ridge                  8.267236
78         West Town                  8.379916
79          Woodlawn                 30.183503

[80 rows x 2 columns]


In [2]:
# Neighborhoods with highest park coverage percentage

top_values = park_coverage.sort_values(by='park_coverage_percentage', ascending=False).head(10)
top_values

,Neighborhood Name,park_coverage_percentage
57,Oakland,36.034773
71,Washington Park,35.927575
79,Woodlawn,30.183503
50,Near South Side,29.694215
43,Loop,29.178687
35,Hyde Park,28.412914
16,Chicago Lawn,27.294109
69,Uptown,22.984600
40,Lincoln Park,20.055056
38,Kenwood,18.993595


In [5]:
bottom_values = park_coverage.sort_values(by='park_coverage_percentage', ascending=True).head(10)
bottom_values

,Neighborhood Name,park_coverage_percentage
58,O'Hare,0.018628
74,West Garfield Park,0.414609
11,Brighton Park,0.459663
32,Hegewisch,0.573258
34,Humboldt Park,0.612775
7,Avondale,0.691102
67,Little Village,0.869194
62,Riverdale,1.032967
66,South Deering,1.059235
37,Jefferson Park,1.119730


In [4]:
treatment_names = ['Cicero', 'Berwyn', 'Oak Park', 'Little Village', 'Humboldt Park', 'Lawndale', 
                   'East Garfield Park', 'West Garfield Park', 'West Town', 'Near West Side', 'Pilsen']
treatment_df = park_coverage[park_coverage['Neighborhood Name'].isin(treatment_names)] 

treatment_df.sort_values(by='park_coverage_percentage', ascending=False)

,Neighborhood Name,park_coverage_percentage
21,East Garfield Park,14.797695
54,Lawndale,8.695658
78,West Town,8.379916
59,Oak Park,2.691841
44,Pilsen,1.995715
51,Near West Side,1.787781
18,Cicero,1.559403
13,Berwyn,1.330999
67,Little Village,0.869194
34,Humboldt Park,0.612775
